In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install tqdm

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OrdinalEncoder,LabelEncoder, OneHotEncoder, PolynomialFeatures, MinMaxScaler,StandardScaler, RobustScaler, Normalizer, QuantileTransformer, PowerTransformer, SplineTransformer
from sklearn.linear_model import LogisticRegression, BayesianRidge, Ridge, Lasso 
from sklearn.compose import make_column_transformer, make_column_selector, ColumnTransformer, TransformedTargetRegressor
from sklearn.metrics import mean_absolute_percentage_error, make_scorer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

import category_encoders as ce

from tqdm import tqdm

### Область работы 2 (выполнение лучшей модели)

In [4]:
# Данный блок(и) в области 2 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [5]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок

In [6]:
# Блок(и) обучения и поверки модели

In [7]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

### Проверка на дупликаты

In [8]:
display(len(df_train) - len(df_train.drop_duplicates()))
df_train.drop_duplicates(inplace=True)
display(len(df_train) - len(df_train.drop_duplicates()))

192

0

### Удаление выбросов

In [9]:
set_index_for_drop = set()
for column in df_train.columns[6:-1]:
    Q1 = df_train[column].quantile(0.25)
    Q3 = df_train[column].quantile(0.75)
    IQR = Q3 - Q1
    indexes = list(df_train[(df_train[column] < Q1-1.5*IQR ) | (df_train[column] > Q3+1.5*IQR)].index)
    set_index_for_drop.update(indexes)

display(df_train.shape)
df_not_outliers = df_train.drop(index=set_index_for_drop).copy()
display(df_not_outliers.shape)

(67406, 12)

(61946, 12)

In [10]:
#Разделяем на X и y
X, y = df_not_outliers.drop(columns=['total_sales_price']), df_not_outliers['total_sales_price']

# Конструируем трансформер для предсказания нашей целевой переменной ('total_sales_price')

In [11]:
axis_transformer = Pipeline(steps=[
    ('polynom', PolynomialFeatures(3,include_bias=False)),
    ('scaler', MinMaxScaler())])

missing = make_column_transformer(
    (KNNImputer(missing_values=0, n_neighbors=3, weights="uniform"), ['size', 'depth_percent', 'table_percent', 'meas_length','meas_width','meas_depth']),
    remainder='passthrough'
     ).set_output(transform='pandas')

axis_features = ['knnimputer__meas_length', 'knnimputer__meas_width', 'knnimputer__meas_depth']
num_features = ['knnimputer__size','knnimputer__depth_percent','knnimputer__table_percent']
ct = ColumnTransformer([
    ("pol_std", axis_transformer, axis_features),
      ('scale', Normalizer(), num_features),
      ('onehot',
      OneHotEncoder(),
      make_column_selector(dtype_include='object'))
])


model_pipe_with_transformed_Y = TransformedTargetRegressor(
    regressor = Pipeline([
        ('missing', missing),
        ('preprocessing', ct),
        ('estimator', KNeighborsRegressor(n_neighbors=6, p=2, leaf_size=5, weights='distance'))
    ]),
    transformer=PowerTransformer(method='box-cox')
)

### Fit and predict target variable with best model

In [12]:
model_pipe_with_transformed_Y.fit(X, y)

TransformedTargetRegressor(regressor=Pipeline(steps=[('missing',
                                                      ColumnTransformer(remainder='passthrough',
                                                                        transformers=[('knnimputer',
                                                                                       KNNImputer(missing_values=0,
                                                                                                  n_neighbors=3),
                                                                                       ['size',
                                                                                        'depth_percent',
                                                                                        'table_percent',
                                                                                        'meas_length',
                                                                                        'meas_width',
                                                                                        'meas_depth'])])),
                                                     ('preprocessing',
                                                      ColumnTransformer(transformers=[('pol_std',
                                                                                       Pipeline(steps=[('polynom',
                                                                                                        P...
                                                                                        'knnimputer__meas_depth']),
                                                                                      ('scale',
                                                                                       Normalizer(),
                                                                                       ['knnimputer__size',
                                                                                        'knnimputer__depth_percent',
                                                                                        'knnimputer__table_percent']),
                                                                                      ('onehot',
                                                                                       OneHotEncoder(),
                                                                                       <sklearn.compose._column_transformer.make_column_selector object at 0x0000022674E862E0>)])),
                                                     ('estimator',
                                                      KNeighborsRegressor(leaf_size=5,
                                                                          n_neighbors=6,
                                                                          weights='distance'))]),
                           transformer=PowerTransformer(method='box-cox'))

In [13]:
# Блок предсказания с использованием тестового набора

In [14]:
final_pred = model_pipe_with_transformed_Y.predict(df_test)
final_pred

array([2111.14645961,  887.50754854, 1791.13092355, ..., 1259.09171333,
       2937.03653214, 1592.11738977])

In [15]:
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = final_pred